In [3]:
!pip install ultralytics pycocotools -q

In [4]:
import os
import csv
import json
import yaml
import random
import shutil
import time
from datetime import datetime

import numpy as np 
import pandas as pd 

import torch

from pycocotools.coco import COCO
from ultralytics import YOLO
import ultralytics
from ultralytics.data.converter import convert_coco

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
SEED = 42
EPOCHS = 50
BATCH_SIZE = 128
IMAGE_SIZE = 512
PROJECT_NAME = "yolo8n_pt_512_coco_full"
BASE_MODEL = "yolov8n.pt"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 20

INPUT_DATASET_ROOT = "/kaggle/input/coco-train/coco"
DATASET_ROOT = "/kaggle/working/dataset"
TRAINING_ROOT = "/kaggle/working/training"

PATHS = {
    'input_root': INPUT_DATASET_ROOT,
    'dataset_root': DATASET_ROOT,
    'training_root': TRAINING_ROOT,
    'train_images': os.path.join(INPUT_DATASET_ROOT, "train"),
    'val_images': os.path.join(INPUT_DATASET_ROOT, "val"),
    'train_annotations': os.path.join(INPUT_DATASET_ROOT, "annotations", "instances_train.json"),
    'val_annotations': os.path.join(INPUT_DATASET_ROOT, "annotations", "instances_val.json"),
    'yaml_file': os.path.join(DATASET_ROOT, "yolo_main.yaml"),
    'trained_model': os.path.join(TRAINING_ROOT, PROJECT_NAME, "weights", "best.pt"),
}

print("The configuration is set:")
print(f"  - EPOCHS: {EPOCHS}")
print(f"  - BATCH_SIZE: {BATCH_SIZE}")
print(f"  - IMAGE_SIZE: {IMAGE_SIZE}")
print(f"  - DEVICE: {DEVICE}")
print(f"  - BASE_MODEL: {BASE_MODEL}")

The configuration is set:
  - EPOCHS: 50
  - BATCH_SIZE: 128
  - IMAGE_SIZE: 512
  - DEVICE: cuda
  - BASE_MODEL: yolov8n.pt


In [6]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

ultralytics.checks()

Ultralytics 8.3.167 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.3/8062.4 GB disk)


In [7]:
convert_coco(
    labels_dir = os.path.join(PATHS['input_root'], "annotations"),
    save_dir = PATHS['dataset_root'],
    use_segments = False,
    use_keypoints = False,
    cls91to80 = False,
    lvis = False,
)

Annotations /kaggle/input/coco-train/coco/annotations/instances_train.json: 100%|██████████| 19500/19500 [00:02<00:00, 6872.11it/s]
Annotations /kaggle/input/coco-train/coco/annotations/instances_val.json: 100%|██████████| 5789/5789 [00:00<00:00, 6883.61it/s]

COCO data converted successfully.
Results saved to /kaggle/working/dataset


In [8]:
print("Loading COCO ...")

train_coco = COCO(PATHS['train_annotations'])
val_coco = COCO(PATHS['val_annotations'])

Loading COCO ...
loading annotations into memory...
Done (t=1.87s)
creating index...
index created!
loading annotations into memory...
Done (t=0.68s)
creating index...
index created!


In [9]:
# Getting all categories
train_categories = train_coco.loadCats(train_coco.getCatIds())
val_categories = val_coco.loadCats(val_coco.getCatIds())

all_categories = {}
for cat in train_categories + val_categories:
    all_categories[cat['id']] = cat['name']

print(f"Found {len(all_categories)}\n{all_categories}")

yaml_categories = {}
for key, value in all_categories.items():
    yaml_categories[key - 1] = value
print(f"Categories for YOLO {yaml_categories}")

Found 3
{1: 'person', 2: 'pet', 3: 'car'}
Categories for YOLO {0: 'person', 1: 'pet', 2: 'car'}


In [11]:
def get_image_filenames_from_coco(coco_obj):
    """Get a list of file names from a COCO object"""
    image_ids = coco_obj.getImgIds()
    filenames = []
    for img_id in image_ids:
        img_info = coco_obj.loadImgs(img_id)[0]
        filename = os.path.splitext(img_info['file_name'])[0]
        filenames.append(filename)
    return filenames

# Getting lists of files
train_files = get_image_filenames_from_coco(train_coco)
val_files = get_image_filenames_from_coco(val_coco)

print(f"Dataset statistics:")
print(f"  - Train images: {len(train_files)}")
print(f"  - Val images: {len(val_files)}")

Dataset statistics:
  - Train images: 22500
  - Val images: 6789


In [12]:
def create_symlinks(image_list, src_dir, dst_dir):
    """Creationg simlinks"""
    os.makedirs(dst_dir, exist_ok=True)
    created_count = 0
    
    for filename in image_list:
        src_path = os.path.join(src_dir, filename + ".jpg")
        dst_path = os.path.join(dst_dir, filename + ".jpg")
        
        if not os.path.exists(src_path):
            continue
            
        if not os.path.exists(dst_path):
            os.symlink(src_path, dst_path)
            created_count += 1
    
    print(f"Created {created_count} symlinks в {dst_dir}")
    return created_count

# Create symlinks for images
print("Create symbolic links...")

train_symlinks = create_symlinks(train_files, PATHS['train_images'], 
                                os.path.join(PATHS['dataset_root'], 'images', 'train'))
val_symlinks = create_symlinks(val_files, PATHS['val_images'], 
                              os.path.join(PATHS['dataset_root'], 'images', 'val'))

print(f"Symbolic links created:")
print(f" - Train: {train_symlinks}")
print(f" - Val: {val_symlinks}")

Create symbolic links...
Created 22500 symlinks в /kaggle/working/dataset/images/train
Created 6789 symlinks в /kaggle/working/dataset/images/val
Symbolic links created:
 - Train: 22500
 - Val: 6789


In [13]:
# Creating a YAML configuration for YOLO
data_yaml = {
    "path": PATHS['dataset_root'],
    "train": "images/train",
    "val": "images/val",
    "names": yaml_categories
}

with open(PATHS['yaml_file'], "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False, allow_unicode=True)

print(f"YAML configuration is saved: {PATHS['yaml_file']}")
print("YAML configuration:")
print(yaml.dump(data_yaml, default_flow_style=False, allow_unicode=True))

YAML configuration is saved: /kaggle/working/dataset/yolo_main.yaml
YAML configuration:
names:
  0: person
  1: pet
  2: car
path: /kaggle/working/dataset
train: images/train
val: images/val



In [14]:
print(f"Model initialization: {BASE_MODEL}")
model = YOLO(BASE_MODEL)
print(f"Model initialized. Device in use: {DEVICE}")

Model initialization: yolov8n.pt


100%|██████████| 6.25M/6.25M [00:00<00:00, 18.9MB/s]

Model initialized. Device in use: cuda


In [15]:
print(f"Training configuration:")
print(f"  - Epochs: {EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Image size: {IMAGE_SIZE}")
print(f"  - Device: {DEVICE}")
print(f"  - Patience: {PATIENCE}")

training_results = model.train(
    data=PATHS['yaml_file'],
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMAGE_SIZE,
    project=PATHS['training_root'],
    name=PROJECT_NAME,
    device=DEVICE,
    seed=SEED,
    patience=PATIENCE,
)

trained_model_path = PATHS['trained_model']
print(f"Model saved: {trained_model_path}")

Training configuration:
  - Epochs: 50
  - Batch size: 128
  - Image size: 512
  - Device: cuda
  - Patience: 20
Ultralytics 8.3.167 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/yolo_main.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8n_pt_512_coco_f

100%|██████████| 755k/755k [00:00<00:00, 3.80MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 16.3MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.3±0.3 ms, read: 47.4±12.1 MB/s, size: 177.2 KB)


...

                   all       6789      27484      0.779      0.634      0.693      0.484

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      10.6G      1.047      0.761      1.068        388        512: 100%|██████████| 176/176 [02:26<00:00,  1.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:31<00:00,  1.15s/it]


                   all       6789      27484      0.782      0.633      0.694      0.484

50 epochs completed in 2.545 hours.
Optimizer stripped from /kaggle/working/training/yolo8n_pt_512_coco_full/weights/last.pt, 6.2MB
Optimizer stripped from /kaggle/working/training/yolo8n_pt_512_coco_full/weights/best.pt, 6.2MB

Validating /kaggle/working/training/yolo8n_pt_512_coco_full/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:36<00:00,  1.34s/it]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       6789      27484      0.782      0.633      0.694      0.484
                person       3870      16380      0.805      0.573       0.68      0.443
                   pet       1804       2238      0.747      0.779      0.748      0.575
                   car       2455       8866      0.794      0.547      0.653      0.436
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /kaggle/working/training/yolo8n_pt_512_coco_full
Model saved: /kaggle/working/training/yolo8n_pt_512_coco_full/weights/best.pt


In [17]:
def create_training_readme():
    """Create a comprehensive README for the training results"""
    
    # Get training results path
    results_path = os.path.join(PATHS['training_root'], PROJECT_NAME)
    readme_path = os.path.join(results_path, 'README.md')
    
    # Read results if available
    results_csv = os.path.join(results_path, 'results.csv')
    
    readme_content = f"""# YOLO Training Results - {PROJECT_NAME}
## Training Configuration
**Model:** {BASE_MODEL}  
**Project:** {PROJECT_NAME}  
**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
### Hyperparameters
- **Epochs:** {EPOCHS}
- **Batch Size:** {BATCH_SIZE}  
- **Image Size:** {IMAGE_SIZE}
- **Device:** {DEVICE}
- **Patience:** {PATIENCE}
- **Seed:** {SEED}
### Dataset Information
- **Dataset Source:** `{INPUT_DATASET_ROOT}`
- **Training Images:** {len(train_files)}
- **Validation Images:** {len(val_files)}
- **Total Images:** {len(train_files) + len(val_files)}
### Class Configuration
**Class Names:**
```yaml
{yaml.dump(yaml_categories, default_flow_style=False, allow_unicode=True)}```
## Training Results
"""
    
    # Add training metrics if results.csv exists
    if os.path.exists(results_csv):
        try:
            df = pd.read_csv(results_csv)
            last_epoch = df.iloc[-1]
            
            readme_content += f"""### Final Metrics (Epoch {int(last_epoch['epoch'])})
- **Train Box Loss:** {last_epoch.get('train/box_loss', 'N/A'):.4f}
- **Train Class Loss:** {last_epoch.get('train/cls_loss', 'N/A'):.4f}
- **Train DFL Loss:** {last_epoch.get('train/dfl_loss', 'N/A'):.4f}
- **Validation mAP50:** {last_epoch.get('metrics/mAP50(B)', 'N/A'):.4f}
- **Validation mAP50-95:** {last_epoch.get('metrics/mAP50-95(B)', 'N/A'):.4f}
- **Validation Precision:** {last_epoch.get('metrics/precision(B)', 'N/A'):.4f}
- **Validation Recall:** {last_epoch.get('metrics/recall(B)', 'N/A'):.4f}
### Training Progress
"""
            
        except Exception as e:
            readme_content += f"*Error loading training results: {str(e)}*\n\n"
    
    # Add file structure and environment info
    readme_content += f"""## Project Structure

## Training Environment
- **Python Version:** 3.11.11
- **PyTorch Version:** {torch.__version__}
- **Ultralytics Version:** {ultralytics.__version__}
- **CUDA Available:** {torch.cuda.is_available()}
---
*Generated automatically on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""
    
    # Write README
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme_content)
    
    print(f"✅ README created: {readme_path}")
    
    # Also create a summary JSON
    summary_data = {
        "project_name": PROJECT_NAME,
        "model": BASE_MODEL,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "image_size": IMAGE_SIZE,
        "device": DEVICE,
        "train_images": len(train_files),
        "val_images": len(val_files),
        "timestamp": datetime.now().isoformat()
    }
    
    summary_path = os.path.join(results_path, 'training_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary_data, f, indent=2)
    
    print(f"✅ Summary JSON created: {summary_path}")
    return readme_path


create_training_readme()

✅ README created: /kaggle/working/training/yolo8n_pt_512_coco_full/README.md
✅ Summary JSON created: /kaggle/working/training/yolo8n_pt_512_coco_full/training_summary.json


'/kaggle/working/training/yolo8n_pt_512_coco_full/README.md'

In [18]:
from IPython.display import display, HTML
import os

os.chdir('/kaggle/working')

zip_filename = f"{PROJECT_NAME}_training.zip"
folder_to_zip = "training"

print(f"📦 Creating archive: {zip_filename}")

!zip -r -q {zip_filename} {folder_to_zip}/

zip_path = f'/kaggle/working/{zip_filename}'

if os.path.exists(zip_path):
    file_size = os.path.getsize(zip_path) / (1024*1024)  # у MB
    print(f"\n✅ Archive created successfully!")
    print(f"📁 File: {zip_filename}")
    print(f"📊 Size: {file_size:.1f} MB")
    print(f"📍 Path: {zip_path}")
    
    print(f"\n📋 Archive contents:")
    !zipinfo {zip_filename} | head -20
    
    display(HTML(f'''
    <div style="background-color: #e8f5e8; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h3>📥 Download Ready</h3>
        <a href="{zip_filename}" download style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; font-weight: bold;">
            📥 Download {zip_filename} ({file_size:.1f} MB)
        </a>
    </div>
    '''))
else:
    print("❌ Error: Archive not created!")
    print("📁 Checking working directory contents:")
    !ls -la /kaggle/working/

📦 Creating archive: yolo8n_pt_512_coco_full_training.zip

✅ Archive created successfully!
📁 File: yolo8n_pt_512_coco_full_training.zip
📊 Size: 17.4 MB
📍 Path: /kaggle/working/yolo8n_pt_512_coco_full_training.zip

📋 Archive contents:
Archive:  yolo8n_pt_512_coco_full_training.zip
Zip file size: 18251037 bytes, number of entries: 30
drwxr-xr-x  3.0 unx        0 bx stor 25-Jul-16 12:52 training/
drwxr-xr-x  3.0 unx        0 bx stor 25-Jul-16 15:28 training/yolo8n_pt_512_coco_full/
-rw-r--r--  3.0 unx   180690 bx defN 25-Jul-16 15:27 training/yolo8n_pt_512_coco_full/BoxR_curve.png
-rw-r--r--  3.0 unx   547609 bx defN 25-Jul-16 15:26 training/yolo8n_pt_512_coco_full/val_batch2_labels.jpg
-rw-r--r--  3.0 unx   181732 bx defN 25-Jul-16 12:53 training/yolo8n_pt_512_coco_full/labels.jpg
-rw-r--r--  3.0 unx     1063 tx defN 25-Jul-16 15:28 training/yolo8n_pt_512_coco_full/README.md
-rw-r--r--  3.0 unx   364073 bx defN 25-Jul-16 15:26 training/yolo8n_pt_512_coco_full/val_batch0_labels.jpg
-rw-r--